In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [2]:
imdb_data=pd.read_csv(r'D:\駿燊\nlpPractice\IMDB Dataset.csv', encoding="unicode_escape")
print(imdb_data.shape)
imdb_data.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [3]:
imdb_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [4]:
imdb_data.iloc[1,0]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [5]:
#split the dataset  
#train dataset
train_reviews=imdb_data.review[:40000]
train_sentiments=imdb_data.sentiment[:40000]
#test dataset
test_reviews=imdb_data.review[40000:]
test_sentiments=imdb_data.sentiment[40000:]
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

(40000,) (40000,)
(10000,) (10000,)


In [6]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

### Removing html strips and noise text
```ruby
data_soup = BeautifulSoup('<div data-foo="value">foo!</div>', 'html.parser')
data_soup.get_text()
'foo!'
```
```ruby
inputStr = "hello 123 world 456"
replacedStr = re.sub("\d+", "222", inputStr)
'hello 222 world 222'
```

In [7]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(denoise_text)

C:\Users\DBLAB2020\anaconda3\envs\nlpPractice\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [8]:
imdb_data.iloc[1,0]

'A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well done.'

### Removing special characters

In [9]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_special_characters)

In [10]:
imdb_data.iloc[1,0]

'A wonderful little production The filming technique is very unassuming very oldtimeBBC fashion and gives a comforting and sometimes discomforting sense of realism to the entire piece The actors are extremely well chosen Michael Sheen not only has got all the polari but he has all the voices down pat too You can truly see the seamless editing guided by the references to Williams diary entries not only is it well worth the watching but it is a terrificly written and performed piece A masterful production about one of the great masters of comedy and his life The realism really comes home with the little things the fantasy of the guard which rather than use the traditional dream techniques remains solid then disappears It plays on our knowledge and our senses particularly with the scenes concerning Orton and Halliwell and the sets particularly of their flat with Halliwells murals decorating every surface are terribly well done'

### Text stemming
```ruby
ps=nltk.porter.PorterStemmer()
ps.stem('RulE')
>>rule
```

In [11]:
#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(simple_stemmer)

In [12]:
imdb_data.iloc[1,0]

'a wonder littl product the film techniqu is veri unassum veri oldtimebbc fashion and give a comfort and sometim discomfort sens of realism to the entir piec the actor are extrem well chosen michael sheen not onli ha got all the polari but he ha all the voic down pat too you can truli see the seamless edit guid by the refer to william diari entri not onli is it well worth the watch but it is a terrificli written and perform piec a master product about one of the great master of comedi and hi life the realism realli come home with the littl thing the fantasi of the guard which rather than use the tradit dream techniqu remain solid then disappear it play on our knowledg and our sens particularli with the scene concern orton and halliwel and the set particularli of their flat with halliwel mural decor everi surfac are terribl well done'

### Removing stopwords

In [13]:
tokens = tokenizer.tokenize(imdb_data['review'].iloc[1])
tokens

['a',
 'wonder',
 'littl',
 'product',
 'the',
 'film',
 'techniqu',
 'is',
 'veri',
 'unassum',
 'veri',
 'oldtimebbc',
 'fashion',
 'and',
 'give',
 'a',
 'comfort',
 'and',
 'sometim',
 'discomfort',
 'sens',
 'of',
 'realism',
 'to',
 'the',
 'entir',
 'piec',
 'the',
 'actor',
 'are',
 'extrem',
 'well',
 'chosen',
 'michael',
 'sheen',
 'not',
 'onli',
 'ha',
 'got',
 'all',
 'the',
 'polari',
 'but',
 'he',
 'ha',
 'all',
 'the',
 'voic',
 'down',
 'pat',
 'too',
 'you',
 'can',
 'truli',
 'see',
 'the',
 'seamless',
 'edit',
 'guid',
 'by',
 'the',
 'refer',
 'to',
 'william',
 'diari',
 'entri',
 'not',
 'onli',
 'is',
 'it',
 'well',
 'worth',
 'the',
 'watch',
 'but',
 'it',
 'is',
 'a',
 'terrificli',
 'written',
 'and',
 'perform',
 'piec',
 'a',
 'master',
 'product',
 'about',
 'one',
 'of',
 'the',
 'great',
 'master',
 'of',
 'comedi',
 'and',
 'hi',
 'life',
 'the',
 'realism',
 'realli',
 'come',
 'home',
 'with',
 'the',
 'littl',
 'thing',
 'the',
 'fantasi',
 'of'

In [14]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_stopwords)

{'between', 'my', 'didn', "hadn't", 'after', "aren't", 'and', 'most', 'he', 'theirs', "don't", 'there', 'very', 'that', 'than', 'his', 'had', 'other', 'her', "that'll", 'mightn', 'be', 'been', 'over', "hasn't", "didn't", "weren't", 'or', "mustn't", 'have', 'in', 'our', 'here', 'too', 'by', 'off', 'did', 'aren', 'hasn', 'who', 'about', 'out', 'ain', "shouldn't", 'how', "haven't", "mightn't", 'up', 'needn', 'won', 'has', 'couldn', 'ourselves', 'a', 'until', 'them', 'o', 'itself', "doesn't", 'own', 'y', 'at', 'were', 't', "you've", 'why', 'if', 'can', 'we', 'when', 'from', 'these', 'll', 'during', 'their', 'wouldn', 'was', 'having', 'herself', 'of', 'myself', 'should', 'but', 'the', 'd', 'mustn', 'me', 'before', 'under', 'more', 'yourself', 'she', 're', "it's", 'it', 'for', 'to', 'whom', "wouldn't", 'yours', "she's", 'its', 'as', "wasn't", "shan't", 'only', 'once', 'themselves', 'further', 'are', 'such', 'hers', 'into', 've', "you'd", 'where', 'which', 'this', 'so', 'then', 'some', 'ma', 

In [15]:
imdb_data.iloc[1,0]

'wonder littl product film techniqu veri unassum veri oldtimebbc fashion give comfort sometim discomfort sens realism entir piec actor extrem well chosen michael sheen onli ha got polari ha voic pat truli see seamless edit guid refer william diari entri onli well worth watch terrificli written perform piec master product one great master comedi hi life realism realli come home littl thing fantasi guard rather use tradit dream techniqu remain solid disappear play knowledg sens particularli scene concern orton halliwel set particularli flat halliwel mural decor everi surfac terribl well done'

### Normalized train reviews

In [16]:
#normalized train reviews
norm_train_reviews=imdb_data.review[:40000]
norm_train_reviews[0]

'one review ha mention watch 1 oz episod youll hook right thi exactli happen meth first thing struck oz wa brutal unflinch scene violenc set right word go trust thi show faint heart timid thi show pull punch regard drug sex violenc hardcor classic use wordit call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home manyaryan muslim gangsta latino christian italian irish moreso scuffl death stare dodgi deal shadi agreement never far awayi would say main appeal show due fact goe show wouldnt dare forget pretti pictur paint mainstream audienc forget charm forget romanceoz doesnt mess around first episod ever saw struck nasti wa surreal couldnt say wa readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard wholl sold nickel inmat wholl kill order get away well manner middl class inmat turn prison bitch due lack street skill prison exp

In [17]:
#Normalized test reviews
norm_test_reviews=imdb_data.review[40000:]
norm_test_reviews[45005]

'read review watch thi piec cinemat garbag took least 2 page find somebodi els didnt think thi appallingli unfunni montag wasnt acm humour 70 inde ani era thi isnt least funni set sketch comedi ive ever seen itll till come along half skit alreadi done infinit better act monti python woodi allen wa say nice piec anim last 90 second highlight thi film would still get close sum mindless drivelridden thi wast 75 minut semin comedi onli world semin realli doe mean semen scatolog humour onli world scat actual fece precursor joke onli mean thi handbook comedi tit bum odd beaver niceif pubesc boy least one hand free havent found playboy exist give break becaus wa earli 70 way sketch comedi go back least ten year prior onli way could even forgiv thi film even made wa gunpoint retro hardli sketch clown subtli pervert children may cut edg circl could actual funni come realli quit sad kept go throughout entir 75 minut sheer belief may save genuin funni skit end gave film 1 becaus wa lower scoreand

### Bags of words model
- It is used to convert text documents to numerical vectors or bag of words.
    - 計算單字出現在文件的次數

#### example

In [18]:
text = ['Hello my name is james',
'james this is my python notebook',
'james trying to create a big dataset',
'james of words to try differnt',
'features of count vectorizer']

In [19]:
coun_vect = CountVectorizer()
count_matrix = coun_vect.fit_transform(text)
count_array = count_matrix.toarray()

In [20]:
df = pd.DataFrame(data=count_array,columns = coun_vect.get_feature_names_out())
df

,big,count,create,dataset,differnt,features,hello,is,james,my,name,notebook,of,python,this,to,try,trying,vectorizer,words
0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,1,1,0,1,0,1,1,0,0,0,0,0
2,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0
3,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,1
4,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0


#### 

In [21]:
#Count vectorizer for bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train_reviews=cv.fit_transform(norm_train_reviews)
#transformed test reviews
cv_test_reviews=cv.transform(norm_test_reviews)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)
#vocab=cv.get_feature_names()-toget feature names

BOW_cv_train: (40000, 6209098)
BOW_cv_test: (10000, 6209098)


In [22]:
cv.get_feature_names_out()

array(['00 1991', '00 1991 wa', '00 act', ...,
       'zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz',
       'zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz 8ooop',
       'zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz 8ooop sorri'], dtype=object)

### Term Frequency-Inverse Document Frequency model (TFIDF)
- It is used to convert text documents to matrix of tfidf features.
TfidfVectorizer = TF + IDF
---
--> TF   - 詞頻（term frequency，tf)

--> IDF - 逆向文檔頻率(inverse document frequency, idf)

- 舉例：

  假設現在有兩個關鍵詞“經濟”，“非常”。

  現在基礎數據是1000篇文章的語料庫。其中1000篇文章“非常”一詞出現的頻率很高，10篇文章“經濟”一詞出現的很高。

- 計算公式：

  TF（詞頻）= 出現的頻率/總次數

  IDF（逆向文檔頻率） = log10 （文件總數/包含該詞語的文件數目）

  TF-IDF = TF * IDF

- 要求計算：

  現有兩篇文章

  文章A（100詞）：10次“經濟” TF-IDF：0.2

    tf：10/100 = 0.1

    idf：lg （1000/10） = 2

  文章B（100詞）：10次“非常” TF-IDF：0.1

    tf：10/100 = 0.1

    idf：lg （1000/100） = 1

In [23]:
#Tfidf vectorizer
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(norm_train_reviews)
#transformed test reviews
tv_test_reviews=tv.transform(norm_test_reviews)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

Tfidf_train: (40000, 6209098)
Tfidf_test: (10000, 6209098)


### Labeling the sentiment text

In [24]:
#labeling the sentient data
lb=LabelBinarizer()
#transformed sentiment data
sentiment_data=lb.fit_transform(imdb_data['sentiment'])
print(sentiment_data.shape)

(50000, 1)


In [25]:
#Spliting the sentiment data
train_sentiments=sentiment_data[:40000]
test_sentiments=sentiment_data[40000:]
print(train_sentiments)
print(test_sentiments)

[[1]
 [1]
 [1]
 ...
 [1]
 [0]
 [0]]
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


### Modelling the dataset

In [27]:
import torch
import torch.nn as nn

#### logistic regression
- 1) Design model (input_size, output_size, forward pass)
- 2) Construst loss and optimizer
- 3) Training loop
    - forward pass: compute prediction and loss
    - backward pass: gardients
    - update weights

In [84]:
cv_train_reviews.shape, cv_test_reviews.shape

((40000, 6209098), (10000, 6209098))

In [78]:
train_sentiments_torch = torch.from_numpy(train_sentiments.astype(np.float32))
test_sentiments_torch = torch.from_numpy(test_sentiments.astype(np.float32))

In [113]:
train_sentiments_torch = train_sentiments_torch.view(train_sentiments_torch.shape[0], 1)
test_sentiments_torch = test_sentiments_torch.view(test_sentiments_torch.shape[0], 1)

In [114]:
train_sentiments_torch.shape

torch.Size([40000, 1])

In [62]:
n_features = cv_train_reviews.shape[1]

In [99]:
class LogisticRegression(nn.Module):

    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

model = LogisticRegression(n_features)

#
learning_rate = 0.1
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    y_predicted= model(cv_train_reviews_tensor.float())
    loss = criterion(y_predicted, train_sentiments_torch)

    loss.backward()

    optimizer.step()

    optimizer.zero_grad()

    if (epoch+1) % 1 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

In [ ]:
# because we are done for training loop, we don't need gradient
with torch.no_grad():
    y_predicted = model(cv_train_reviews_tensor.float())
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(test_sentiments_torch).sum() / float(cv_test_reviews_tensor.shape[0])
    print(f'accuracy = {acc:.4f}')